In [1]:
!pip install earthengine-api

In [2]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=cVnoU7Rm1bKk3fNmSMWY_LmApWnnYJtD1d2rVvnaNTA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhiKgE7Ldff7d8z63OHq2aF2pbvh6hZKipUuf4L9WHdZGnPH0t8pD8

Successfully saved authorization token.


In [3]:
# Initialize the library.
ee.Initialize()

In [4]:
## set up Data
SoCal = ee.FeatureCollection("users/sabenz/allStations_sep2020") # I've uploaded this shapefile to GEE. It's all of our stations in CA
Gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filterDate('2019-1-1','2020-6-1').filter(ee.Filter.calendarRange(1,5,'month'))  

stations = SoCal.distinct('site_ID').select(['site_ID'],['site']); # some stations were double, so this takes care of that.

def getmean(image):
    temp = image.select('tmmn').add(image.select('tmmx')).divide(2).subtract(273.15).rename('Temp') # get mean temp from max and min
    RH = image.select('rmin').add(image.select('rmax')).divide(2).rename('RH')# get mean relative humidity from min and max
    return image.addBands([temp,RH])

Gridmet = Gridmet.map(getmean) # apply the fundtion get mean to each image in the gridmet image collection
Gridmet = Gridmet.select(['pr','Temp','RH'])

mysize = stations.size().getInfo();
mystations = stations.aggregate_array('site').getInfo()
print('Stations:',mystations);
print('# of Stations:',mysize);

Stations: ['PurpleAir34103', 'PurpleAir4103', 'PurpleAir18907', 'PurpleAir40779', 'PurpleAir22675', 'PurpleAir34037', 'PurpleAir6720', 'PurpleAir3271', 'PurpleAir43019', 'PurpleAir36453', 'PurpleAir27147', 'PurpleAir2673', 'PurpleAir19591', 'PurpleAir33505', 'PurpleAir14311', 'PurpleAir49947', 'PurpleAir46941', 'PurpleAir33122', 'PurpleAir51677', 'PurpleAir12869', 'PurpleAir28529', 'PurpleAir19471', 'PurpleAir22067', 'PurpleAir38479', 'PurpleAir27359', 'PurpleAir31533', 'PurpleAir34735', 'PurpleAir51457', 'PurpleAir17827', 'PurpleAir42129', 'CARB3209', 'PurpleAir20707', 'PurpleAir22883', 'PurpleAir22471', 'PurpleAir15165', 'PurpleAir18679', 'PurpleAir19351', 'PurpleAir15653', 'PurpleAir42265', 'PurpleAir37949', 'PurpleAir31081', 'PurpleAir27101', 'PurpleAir4974', 'PurpleAir31155', 'CARB2221', 'PurpleAir15235', 'PurpleAir1806', 'PurpleAir3909', 'PurpleAir34135', 'PurpleAir20757', 'PurpleAir20547', 'PurpleAir38829', 'PurpleAir44367', 'PurpleAir16659', 'PurpleAir21353', 'PurpleAir4762', '

In [5]:
# get climate data for all locations
def getsamples(image):
    time=ee.Date(image.get('system:time_start'))
    myyear = time.get('year')
    mymonth = time.get('month')
    myday = time.get('day')
    feature = image.reduceRegions(stations, ee.Reducer.mean(), 30)
    feature = feature.map(lambda ft: ft.set('year',myyear,'month',mymonth,'day',myday))
    return feature

res = Gridmet.map(getsamples)
res = ee.FeatureCollection(res).flatten()

print(res.first().getInfo())

task = ee.batch.Export.table.toDrive(collection=res,
                                     description='Climate_perStation',
                                     #folder='CA_tract' #name of subfolder if needed
                                     )
task.start()


{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-122.14053009424156, 37.82880608241337]}, 'id': '20190101_000000000000000005e2', 'properties': {'RH': 52.80000114440918, 'Temp': 4.700006103515648, 'day': 1, 'month': 1, 'pr': 0, 'site': 'PurpleAir34103', 'year': 2019}}
